## Blue Sky Above: Pollution estimation using hyper-spectral satellite imagery and maps

**Submission for the Blue Sky Challenge hosted by IEEE Young Professionals. <br>
Theme:** Blue Sky Above <br>
**Team:** Diyaz

**Solution overview:**<br>
>The goal is to estimate NO2 levels at a place at a specified time using satellite measurements from upto 7 prior days and map data.
This must be done using minimal satellite data while maximizing accuracy. Furthermore, it needs to minimize data download requirement and processing needs to be able to run in a mobile device. <br>

> Our solution assumes that 7 days of satellite data is stored in a server which takes mobile app requests through an API.<br>
The server side program uses an autoencoder network to compress the <u> most recent satellite data</u>  for the mobile user's location and time which is then downloaded by the app. <br>
Further, the app downloads a map centred at the location of the phone of an area of dimensions 7x7 sq.km to extract land-use features.<br>
This gathered data along with the time difference between the satellite measurement and current time is fed to the app's NO2 estimator. <br>
The NO2 estimator is a Random Forest Regressor using just 10 trees of depth 20 - this can execute on all modern mobile phones. 
The estimator processes the data and provides the current NO2 estimate in µg/m^3.

### Installing Necessary Libraries/packages

In [2]:
import sys
from IPython.display import clear_output

In [ ]:
# ------------------------------------------------------------------------------------------------------------------------------------
# Install a pip package in the current Jupyter kernel
# ------------------------------------------------------------------------------------------------------------------------------------

!{sys.executable} -m pip install numpy
!{sys.executable} -m pip install smopy
!{sys.executable} -m pip install sentinelsat
!{sys.executable} -m pip install sklearn
!{sys.executable} -m pip install opencv-python
!{sys.executable} -m pip install geopy
!{sys.executable} -m pip install xarray
!{sys.executable} -m pip install tqdm
!{sys.executable} -m pip install tensorflow
clear_output(wait = True)
print('All necessary modules are installed!')

##### Code timing function

In [9]:
# ------------------------------------------------------------------------------------------------------------------------------------
# (Code provided in the BlueSkyAbove getting-started notebook.)
# ------------------------------------------------------------------------------------------------------------------------------------

import time

def TicTocGenerator():
    # Generator that returns time differences
    ti = 0           # initial time
    tf = time.time() # final time
    while True:
        ti = tf
        tf = time.time()
        yield tf-ti # returns the time difference

TicToc = TicTocGenerator() # create an instance of the TicTocGen generator

# This will be the main function through which we define both tic() and toc()
def toc(tempBool=True):
    # Prints the time difference yielded by generator instance TicToc
    tempTimeInterval = next(TicToc)
    if tempBool:
        print( "Elapsed time: %f seconds.\n" %tempTimeInterval )

def tic():
    # Records a time in TicToc, marks the beginning of a time interval
    toc(False)

### Downloading the hyperspectral satellite data
For reliable functioning of the NO2 estimation model, satellite data upto 7 days prior to the time of estimation needs to be available.
The following code segment will use the SentinelAPI to get information about the available data products for the required location during the a specified date range. <br> **Note:** When prompted to enter the 'start date', enter a date at least 3 days prior to the date when the first estimation is required.
<br>_Satellite data will be downloaded and stored in the folder data/L2_

In [3]:
import Sentinel5PDataDownload as s5pdd

# ------------------------------------------------------------------------------------------------------------------------------------
# By default satellite data is downloaded for the following specifications:
# >>> Dates: 25-03-2021 to 05-04-2021
# >>> Area: Area: enclosed in latitudes: [51.25,51.75] and longitudes: [-0.6, -0.28]
# (These are the same specifications as provided in the BlueSkyAbove getting-started notebook.)
# ------------------------------------------------------------------------------------------------------------------------------------

startdate, enddate, LatMin, LatMax, LngMin, LngMax = '20210325', '20210405', 51.25, 51.75, -0.6, 0.28

c = input('Enter new specifications (Y/N)?')

if c in ['y','Y','yes','Yes']:
    startdate = input('Enter start date (YYYYMMDD format): ')
    enddate   = input('Enter end date (YYYYMMDD format): ')
    LatMin, LatMax, LngMin, LngMax = map(float, input('Enter minLat, maxLat, minLong, maxlong separated by commas: ').split())
else:
    clear_output(wait = True)
    print('Continuing with default specifications...')
    
# ------------------------------------------------------------------------------------------------------------------------------------
# The following lines will use the SentinelAPI to download metadata about the available data products for the specified dates and area.
# The names of the data product files are stored in satellite_data_filenames.csv 
# ------------------------------------------------------------------------------------------------------------------------------------

products_found = s5pdd.getProductList(startdate, enddate, LatMin, LatMax, LngMin, LngMax)
s5pdd.saveProductNames(products_found)

Continuing with default specifications...
satellite_data_filenames.csv has been updated.


Start downloading satellite data. <br>
**Note:** Copy the satellite data files in data/L2 folder of the current project repository to avoid re-downloading. <br>
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;Skip the next code cell after copying the files.

In [ ]:
s5pdd.downloadNewProducts(products_found)

### Loading the trained model

In [4]:
import os.path
import numpy as np
import pandas as pd
import mapDataProcess
import satelliteDataProcess
from pickle import load
from datetime import datetime
from tensorflow.keras.models import load_model

In [5]:
data_scaler = load(open('models/satellite_data_scaler.pkl', 'rb'))
data_compressor = load_model('satellite_data_encoder')
model = load(open('models/regressor_forest.pkl', 'rb'))
satelliteDataFiles = pd.read_csv('data/functional_data/satellite_data_filenames.csv')
sites = pd.read_csv('data/functional_data/Ground_Data_Sites.csv')
color_range = pd.read_csv('data/functional_data/color_ranges.csv')
color_range['Low'] = color_range.apply(lambda lows: np.array([int(x) for x in lows['Low'].strip('][').split(' ') if x!='']), axis=1)
color_range['High'] = color_range.apply(lambda lows: np.array([int(x) for x in lows['High'].strip('][').split(' ') if x!='']), axis=1)

For the estimation of NO2, map data is downloaded from OpenStreetMap server to extract land-use data. <br>
As multiple time instances will be evaluated for each location, the map data will be reused. To avoid having to repeatedly download the same map data, every new map data will be stored in a dictionary. <br>
This can be replicated in a mobile app by saving user data for the most frequent locations - eg. the user's residence.

In [6]:
land_use_dict = dict() #dictionary to save land_use data extracted from OpenStreetMap data

#### NO2 Estimation Function
The function accepts a latitude, longitude and datetime as input and returns an estimate of the NO2 concentration for the specified time and place in µg/m^3. <br>
* Dependencies:
>* satelliteDataProcess.py <br>
>* mapDataProcess.py <br>
>* Pandas <br>
>* Numpy
* Parameters:
>* latitude: Latitude of the place of interest <br>
>* longitude: Longitude of the place of interest <br>
>* datetime: Date and time of when the NO2 estimate is required as a datetime object <br>
>* data_files: Iterable list of names of Sentinel5P product files available in data/L2 folder. <br>
>* data_compressor: Autoencoder trained to compress the extracted satellite data. <br>
>* data_scaler: MinMaxScaler trained to scale the satellite data before encoding. <br>
>* color_range: Dataframe containing color ranges to filter out features from OpenStreetMap images.
>* model: Trained ML Model for estimating NO2 using satellite and map data.
* Returns:
>* Estimate of the NO2 concentration as float.<br>
>  In case of no estimate, returns None.

In [7]:
def estimateNO2(latitude, longitude, datetime, data_files, data_compressor, data_scaler, color_range, model):   
    data_NA = 0
    try:
        sat_data = satelliteDataProcess.getMostRecentSatelliteData((latitude,longitude), datetime, data_files, data_compressor, data_scaler)
    except:
        data_NA = 1
        print('No recent usable satellite data!')
        
    if(data_NA == 0):
        if ((latitude,longitude) in land_use_dict.keys()):
            land_use = pd.DataFrame(land_use_dict[(latitude,longitude)]).transpose()
            land_use.reset_index(drop=True,inplace=True)
        else:
            land_use = mapDataProcess.getLandUseTerms((latitude,longitude),color_range,km=7)
            land_use.rename(index = {0:(latitude,longitude)}, inplace = True)
            land_use_dict.update(dict(land_use.transpose()).items())
            land_use.reset_index(drop=True,inplace=True)
            
        variables = pd.DataFrame({'Latitude':latitude,'Longitude':longitude},index=[0])
        variables = pd.concat([variables,sat_data,land_use], axis=1)
        variables[['Month','WeekDay','Hour']] = [datetime.month, datetime.weekday(), datetime.hour]
        variables = np.array(variables)

        return model.predict(variables)[0]
    else:
        return None

### NO2 Estimation
This code segment is to be used to get NO2 estimates. <br>
_The first estimate may take longer than the one's after for a place depending on Internet speed due to map download._  

In [10]:
latitude = float(input('Enter the latitude of the location: '))
longitude = float(input('Enter the longitude of the location: '))
datetime = datetime.strptime(input('Enter when to do the estimate (format like: April 4 2021 1:33PM): '), '%B %d %Y %I:%M%p')

tic()
no2 = estimateNO2(latitude, longitude, datetime, satelliteDataFiles['ProductFiles'], data_compressor, data_scaler, color_range, model)
toc()

filesize = os.path.getsize('Compressed_Satellite_Data.csv')/1024
if(not no2): 
    print('No estimate.')
else:
    print('NO2 concentration: %.3f µg/m^3'%no2)
print('Compressed data filesize: %.2f kB'%filesize)

Enter the latitude of the location:  51.4
Enter the longitude of the location:  0.1
Enter when to do the estimate (format like: April 4 2021 1:33PM):  April 1 2021 2:45PM


Elapsed time: 1.255498 seconds.

NO2 concentration: 10.656 µg/m^3
Compressed data filesize: 0.12 kB
